In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import LocalOutlierFactor
import pandas as pd

In [2]:
import pickle
# Load data
with open('../pkl/global_Aopt_array.pkl', 'rb') as f:
    Aopt_array = pickle.load(f)

with open('../pkl/global_Copt_array.pkl', 'rb') as f:
    Copt_array = pickle.load(f)
    
with open('../pkl/global_Bstar.pkl', 'rb') as f:
    Bstar = pickle.load(f)
    
with open('../pkl/global_travel_tensor.pkl', 'rb') as f:
    travel_tensor = pickle.load(f) 

FileNotFoundError: [Errno 2] No such file or directory: '../pkl/global_Aopt_array.pkl'

In [ ]:
week_slices = pd.date_range('2016-01-04', periods=200, freq='W')
tensor_slices = []
for i in range(len(week_slices)):
    if i % 4 == 0 :
        tensor_slices.append(week_slices[i].date())

In [ ]:
week_slices

In [ ]:
travel_df = pd.read_csv('../csv/global_travel_df.csv')
countries = travel_df['node'].unique().T

In [ ]:
countries

In [ ]:
X = np.array(np.exp(Aopt_array))[:,3,:]

In [ ]:
# X = np.array(np.exp(Aopt_array))[:,0,:]
# clf = LocalOutlierFactor(n_neighbors=10, contamination=0.1)
# clf.fit(X)
# neg_scores = clf.negative_outlier_factor_
# lof_scores = (np.max(neg_scores) - neg_scores)/(np.max(neg_scores) - np.min(neg_scores))
# # print(lof_scores)
# plt.scatter(tensor_slices[0:-1],lof_scores)

In [ ]:
LOF_by_countries = []
for i in range(Aopt_array[0].shape[0]):
#     X = np.array(np.exp(Aopt_array))[:,i,:]
    X = np.array(Aopt_array)[:,i,:]
    clf = LocalOutlierFactor(n_neighbors=20, contamination=0.1)
    clf.fit(X)
    neg_scores = clf.negative_outlier_factor_
#     lof_scores = (np.max(neg_scores) - neg_scores)/(np.max(neg_scores) - np.min(neg_scores))
    lof_scores = neg_scores * -1
    LOF_by_countries.append(lof_scores)
    

In [ ]:
len(np.array(Aopt_array)[:,i,:])

In [ ]:
plt.figure(figsize=[12,12])
plt.subplot(5, 1, 1)
plt.scatter(tensor_slices[0:-1],LOF_by_countries[0])
plt.title('LOF score of ' + countries[0])
# plt.title('LOF score of Germany')

plt.subplot(5, 1, 2)
plt.scatter(tensor_slices[0:-1],LOF_by_countries[3])
# plt.title('LOF score of Italy')
plt.title('LOF score of ' + countries[3])

plt.subplot(5, 1, 3)
plt.scatter(tensor_slices[0:-1],LOF_by_countries[4])
# plt.title('LOF score of United Kingdom')
plt.title('LOF score of ' + countries[4])

plt.subplot(5, 1, 4)
plt.scatter(tensor_slices[0:-1],LOF_by_countries[5])
# plt.title('LOF score of France')
plt.title('LOF score of ' + countries[5])

plt.subplot(5, 1, 5)
plt.scatter(tensor_slices[0:-1],LOF_by_countries[40])
# plt.title('LOF score of Spain')
plt.title('LOF score of ' + countries[40])

plt.subplots_adjust(hspace=0.5)
plt.savefig("global_LOF_with_seasonality.png")
plt.show()

2018
June
In the UK it will be a year since a van and gun attack was launched in central London - eight people were killed. The slaughter at Borough Market had come just weeks after 22 people were left dead by a bombing after a pop concert at Manchester Arena.
https://www.euronews.com/2018/01/01/what-are-the-key-events-to-look-out-for-in-europe-in-2018-

August
It’s a year this month since a group of 12 jihadists launched a series of attacks in and around the Spanish city of Barcelona.
Sixteen people were killed as a result of the violence, including 14 on the Catalan capital’s famous Las Ramblas boulevard.

Later in August it’s half a century since Soviet troops invaded what was then Czechoslovakia to put down the countries brief period of political liberalisation - known as the Prague Spring.

2017 France: Google Trned
https://trends.google.co.kr/trends/explore?date=2017-01-01%202017-12-31&q=france


In [ ]:
# Time Series Analysis
# According to GR, the amount of travel data increase linearly since 2014
# Thus we first start differencing to remove this trend
LOF_DE = LOF_by_countries[5]


# create a differenced series
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return diff
 
# invert differenced forecast
def inverse_difference(last_ob, value):
	return value + last_ob

LOF_dif = difference(LOF_DE)
# len(LOF_dif)

In [ ]:
cont_num = 18
LOF_DE = LOF_by_countries[cont_num]
LOF_dif = difference(LOF_DE)
inverted = [inverse_difference(LOF_DE[i], LOF_dif[i]) for i in range(len(LOF_dif))]
# print(inverted)

plt.figure(figsize=[12,4])
plt.subplot(2, 1, 1)
plt.scatter(tensor_slices[0:-2],inverted)
plt.title('LOF score of '+ countries[cont_num])

plt.subplot(2, 1, 2)
plt.scatter(range(0,len(tensor_slices[0:-2])),LOF_dif)
x = range(0,len(tensor_slices[0:-2]))
y = LOF_dif
z = np.polyfit(x, y, 1)
p = np.poly1d(z)
plt.plot(x,p(x),"r--")
plt.title('differenced LOF score of '+ countries[cont_num])

plt.subplots_adjust(hspace=0.5)
# plt.savefig("Linear_trend.png")

In [ ]:
np.mean(p(x))

In [ ]:
# explore LOF variance by time
LOF_mat=np.array(LOF_by_countries[0:150])
mean_array = np.mean(LOF_mat, axis=0) # mean by time.
np.var(LOF_mat[:,40])
std_array = np.sqrt(np.diag(np.cov(LOF_mat.T)))

In [ ]:
plt.figure(figsize=[12,12])
country_num = 40
plt.subplot(3, 1, 1)
plt.scatter(tensor_slices[0:-1], LOF_by_countries[country_num])
plt.title('LOF score of '+ countries[country_num])

plt.subplot(3, 1, 2)
plt.scatter(tensor_slices[0:-1], np.divide(mean_array,std_array))
# plt.scatter(tensor_slices[0:-1], std_array)
plt.title('Global average LOF')

plt.subplot(3, 1, 3)
plt.scatter(tensor_slices[0:-1], np.divide(LOF_by_countries[country_num]-mean_array, std_array))
plt.title('normalized LOF score of '+ countries[country_num])

plt.subplots_adjust(hspace=0.5)

In [ ]:
LOF_mat[:,:-1].shape
LOF_mat[:,1:].shape
diff_mat = LOF_mat[:,1:] - LOF_mat[:,:-1]
diff_var = np.diag(np.cov(diff_mat.T))
# plt.scatter(range(0,40),difference(LOF_by_countries[2])/np.sqrt(diff_var))
# plt.scatter(range(0,40), np.sqrt(diff_var))
plt.scatter(range(0,len(tensor_slices[0:-2])), difference(LOF_by_countries[10]))
# plt.scatter(range(0,len(tensor_slices[0:-2])), np.log(np.array(LOF_dif)+1))


In [ ]:
with open('global_travel_tensor.pkl', 'rb') as f:
    travel_data = pickle.load(f)

In [ ]:
len(tensor_slices[0:-1])

In [ ]:
# Evaluation
country_num = 4
X2=travel_tensor[:,country_num,:]
diff_travel = difference(np.sum(X2, axis=1))

clf = LocalOutlierFactor(n_neighbors=20, contamination=0.1)
clf.fit(X2)
neg_scores = clf.negative_outlier_factor_
lof_scores = neg_scores * -1

plt.figure(figsize=[12,12])
plt.subplot(3, 1, 1)
# plt.scatter(week_slices, np.sum(X2, axis=1))
plt.plot(week_slices[:], (np.sum(X2, axis=1)))
plt.title('Raw travel traffic of South Korea')

plt.subplot(3, 1, 2)
# plt.scatter(week_slices[1:], difference(lof_scores)).
plt.scatter(tensor_slices[:-1], LOF_by_countries[country_num])
plt.title('LOF scores by TBAD of South Korea')

plt.subplot(3, 1, 3)
plt.scatter(tensor_slices[1:-1], difference(LOF_by_countries[country_num]))
# plt.scatter(tensor_slices[:-1], LOF_by_countries[country_num])
plt.title('Differenced LOF scores by TBAD of South Korea')

plt.subplots_adjust(hspace=0.5)
plt.savefig("LOF_evaluation.png")

In [ ]:
tf = np.abs(difference(LOF_by_countries[country_num]))>0.2

In [ ]:
tf

In [ ]:
# Transport matrix visualization
tensor_slices[-8] # datetime.date(2018, 8, 19)
# print(travel_data[-8,:,0:75])
plt.matshow(travel_data[-6,0:5,0:75])
plt.matshow(travel_data[-7,0:5,0:75])
plt.matshow(travel_data[-8,0:5,0:75])
plt.matshow(travel_data[-9,0:5,0:75])
plt.matshow(travel_data[-10,0:5,0:75])

In [ ]:
len(Aopt_array)

In [ ]:
# A(t) visualization
plt.figure(figsize=[18,5])
# plt.subplot(5, 1, 1)
# plt.figure(figsize=(10,5))
# plt.matshow(Aopt_array[1].T, fignum=1, aspect='auto')
# plt.matshow(Aopt_array[10].T, fignum=2, aspect='auto')
# plt.matshow(Aopt_array[20].T, fignum=3, aspect='auto')
# plt.matshow(Aopt_array[30].T, fignum=4, aspect='auto')
# plt.matshow(Aopt_array[40].T, fignum=5, aspect='auto')

plt.contourf(Aopt_array[1].T); plt.colorbar()
# plt.contourf(Aopt_array[40].T)
# plt.contourf(Aopt_array[20].T, fignum = 3); plt.colorbar()
# plt.contourf(Aopt_array[30].T, fignum = 4); plt.colorbar()
# plt.contourf(Aopt_array[40].T, fignum = 5); plt.colorbar()

In [ ]:
plt.figure(figsize=(12,3))
plt.contourf(Aopt_array[1].T)
plt.colorbar()
plt.show()